In [6]:
# library
import numpy as np
import gff2coverage
import pandas as pd
import csv
%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
path_gff = 'data/results/circos/genes.gff3'
path_genome = 'data/Stuberosum_genome.gff3'
te_classes = ['genes']
unit_value = 1000000

In [8]:
#load chromosomal srtucture
df_genome = pd.read_csv(path_genome, sep='\t', header=None)
df_genome.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']


In [9]:
#load data
df = pd.read_csv(path_gff, sep='\t')
df.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']

In [10]:
# sep by chr
dfs = {}
for seq in df.seqname.unique():
    dfs[seq] = df[df.seqname == seq]

In [11]:
max_chrs = {}
for te_class in te_classes:
    print(te_class)
    out = []
    outcsv = open('data/results/circos/' + te_class + '.csv', 'wb')
    wr = csv.writer(outcsv, quoting=csv.QUOTE_NONE, delimiter='\t')
    for chr_num in range(1, 13):
        chr_name = 'chr' + str(chr_num).zfill(2)
        df_genome_chr = df_genome[df_genome.seqname == chr_name]
        max_chr = df_genome_chr.iloc[0].end
        max_chrs[chr_num] = max_chr
        df_chr = dfs[chr_name]
#        print('%s: elements count: %i max: %i' % (chr_name, len(df_chr.index), max_chr) )
        step = 1 # in mb
        bins = (max_chr / unit_value) 
        x = np.arange(0, bins + step, step)
        y = []
        current = []
        for i in np.arange(0, bins + step, step):
            nt_start = i * unit_value
            nt_end = (i * unit_value) + (unit_value * step) - 1
            if(nt_end > max_chr):
                nt_end = max_chr
            df_res = df_chr[((df_chr.start + df_chr.end) / 2 >= nt_start) & ((df_chr.start + df_chr.end) / 2 <= nt_end)]
            coverage = gff2coverage.calc_coverage_part(df_res, unit_value)
            outlist = [chr_name, nt_start, nt_end, coverage]
            wr.writerow(outlist)
    outcsv.close()


genes
